# Level 1 – Audio Transcription Pipeline

In [1]:
import os
from pathlib import Path
import json
import speech_recognition as sr
import wave
import contextlib

In [2]:
recognizer = sr.Recognizer()

In [3]:
BASE_PATH = Path("sessions")
sessions = sorted([p for p in BASE_PATH.iterdir() if p.is_dir()])


In [4]:
def get_speaker(filename: str) -> str:
    if filename.lower().startswith("ai"):
        return "AI"
    elif filename.lower().startswith("user"):
        return "User"
    return "Unknown"


In [5]:
def get_audio_duration(audio_path: Path):    
    try:
        with contextlib.closing(wave.open(str(audio_path), 'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            return frames / float(rate)
    except:
        return 0.0


In [6]:
def transcribe_audio(audio_path: Path):
    try:
        with sr.AudioFile(str(audio_path)) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(
                audio, 
                language="fa"
            )

        results = [{
            "start": 0.0,
            "end": get_audio_duration(audio_path),
            "text": text.strip()
        }]
        
        return {
            "language": "fa",
            "segments": results
        }
        
    except sr.UnknownValueError:
        return {
            "language": "unknown",
            "segments": [{
                "start": 0.0,
                "end": get_audio_duration(audio_path),
                "text": ""
            }]
        }
    except Exception as e:
        return {
            "language": "error",
            "segments": [{
                "start": 0.0,
                "end": get_audio_duration(audio_path),
                "text": f"خطا: {str(e)}"
            }]
        }


In [7]:
def process_session(session_path: Path):
    transcript = []
    
    audio_files = sorted(session_path.glob("*.wav"))
    
    for audio in audio_files:
        speaker = get_speaker(audio.name)
        result = transcribe_audio(audio)
        
        transcript.append({
            "file": audio.name,
            "speaker": speaker,
            "language": result["language"],
            "segments": result["segments"]
        })
    
    return {
        "session": session_path.name,
        "turns": transcript
    }

In [8]:
all_sessions = []

for session in sessions:
    session_result = process_session(session)
    all_sessions.append(session_result)

In [9]:
with open("level1_transcripts.json", "w", encoding="utf-8") as f:
    json.dump(all_sessions, f, ensure_ascii=False, indent=2)



# Level 2 – Resolution Evaluation


In [10]:
import os
from openai import OpenAI
import json

In [11]:
with open("level1_transcripts.json", "r", encoding="utf-8") as f:
    sessions = json.load(f)


In [12]:
api = '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'

In [13]:


client = OpenAI(
    api_key=api,
    base_url="https://openrouter.ai/api/v1"
)


## Input Description


In [14]:
def dialogue_from_session(session_json):
    lines = []

    for turn in session_json["turns"]:
        speaker = turn["speaker"]
        for seg in turn["segments"]:
            text = seg["text"].strip()
            if text:
                lines.append(f"{speaker}: {text}")

    return "\n".join(lines)


## Resolution Criteria

In [90]:
def build_resolution_prompt(dialogue_text: str) -> str:
    return f"""
شما یک ارزیاب حرفه‌ای تماس‌های استخدامی هستید.

هدف:
تشخیص دهید آیا وضعیت این تماس «تعیین تکلیف شده» است یا خیر.

تعریف قطعی RESOLVED:
تماس «resolved» محسوب می‌شود اگر در پایان مکالمه، اقدام بعدی به‌صورت واضح مشخص شده باشد.
اقدام بعدی می‌تواند شامل موارد زیر باشد:
- تعیین زمان مصاحبه یا مرحله بعد
- موکول شدن تماس به زمان دیگر
- ارجاع تماس به اپراتور انسانی

تماس «resolved نیست» فقط در صورتی که:
- تماس ناگهانی قطع شده باشد
- مکالمه نیمه‌کاره رها شده باشد
- هیچ تصمیم یا اقدام بعدی مشخص نشده باشد

قانون الزامی (خیلی مهم):
وضعیت resolved باید کاملاً با resolution_signal سازگار باشد.

قوانین تطبیق:
- درخواست_اپراتور_انسانی  → resolved = true
- تعیین_تماس_مجدد        → resolved = true
- توافق_مرحله_اول        → resolved = true
- تماس_ناتمام             → resolved = false
- قطع_تماس                → resolved = false

قوانین خروجی:
- فقط و فقط JSON معتبر خروجی بده
- تمام متن‌ها فقط به زبان فارسی باشند
- از هیچ زبان دیگری استفاده نکن
- هیچ توضیحی خارج از JSON ننویس

سیگنال‌های مجاز برای resolution_signal:
- قطع_تماس
- تماس_ناتمام
- درخواست_اپراتور_انسانی
- توافق_مرحله_اول
- تعیین_تماس_مجدد

فرمت خروجی دقیقاً به شکل زیر باشد:
{{
  "resolved": true یا false,
  "confidence": عددی بین 0 و 1,
  "resolution_reason": "توضیح کوتاه و دقیق فارسی",
  "resolution_signal": "یکی از سیگنال‌های مجاز"
}}

متن مکالمه:
{dialogue_text}
"""


In [91]:
def evaluate_resolution(dialogue_text: str):
    prompt = build_resolution_prompt(dialogue_text)
    response = client.chat.completions.create(
        model="qwen/qwen-2.5-7b-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return json.loads(response.choices[0].message.content)


In [92]:
print(
    evaluate_resolution(
        "AI: سلام بابت رزومه تماس گرفتیم\nUser: ممنون، منتظر ایمیل هستم"
    )
)


{'resolved': False, 'confidence': 0.8, 'resolution_reason': 'در این مکالمه، هیچ اقدام بعدی خاصی مشخص نشده است. کاربر تنها منتظر دریافت ایمیل است.', 'resolution_signal': 'تماس_ناتمام'}


In [93]:
level2_results = []

for session in sessions:
    dialogue_text = dialogue_from_session(session)
    resolution = evaluate_resolution(dialogue_text)

    level2_results.append({
        "session": session["session"],
        "resolution": resolution
    })


In [94]:
with open("level2_resolution.json", "w", encoding="utf-8") as f:
    json.dump(level2_results, f, ensure_ascii=False, indent=2)


## Level 3: Call Quality Analysis



In [95]:
with open("level1_transcripts.json", "r", encoding="utf-8") as f:
    level1_sessions = json.load(f)


In [96]:
session_to_dialogue = {}

for s in level1_sessions:
    session_to_dialogue[s["session"]] = dialogue_from_session(s)


In [97]:
def build_quality_prompt(dialogue_text: str) -> str:
    return f"""
You are an HR interview quality evaluator.

STRICT RULES:
- Output ONLY valid JSON
- ALL text MUST be in Persian (Farsi)
- NO markdown
- NO explanations outside JSON
- If unsure, still return JSON

Scoring criteria (0 to 10):
- clarity
- professionalism
- candidate_engagement
- conversation_flow

JSON format:
{{
  "score": number (0-10),
  "criteria": {{
    "clarity": number,
    "professionalism": number,
    "candidate_engagement": number,
    "conversation_flow": number
  }},
  "summary": "Persian paragraph summary"
}}

Transcript:
{dialogue_text}
"""


In [98]:
def analyze_call_quality(dialogue_text: str):
    prompt = build_quality_prompt(dialogue_text)
    response = client.chat.completions.create(
        model="meta-llama/llama-3-8b-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    content = response.choices[0].message.content.strip()

    try:
        return json.loads(content)
    except json.JSONDecodeError:
        return {
            "error": "Invalid JSON returned by model",
            "raw_output": content
        }


In [99]:
print(
    analyze_call_quality(
        "AI: سلام بابت رزومه تماس گرفتیم\nUser: ممنون منتظر ایمیل هستم"
    )
)


{'score': 4, 'criteria': {'clarity': 3, 'professionalism': 4, 'candidate_engagement': 2, 'conversation_flow': 5}, 'summary': 'مصاحبه با متقاضی آغاز شد اما متقاضی به طور کافی پاسخگوی سوالات نبود و در مورد رزومه فقط گفت منتظر ایمیل هستم.'}


In [100]:
level3_results = []

for result in level2_results:
    session_name = result["session"]
    dialogue_text = session_to_dialogue[session_name]

    quality_result = analyze_call_quality(dialogue_text)

    level3_results.append({
        "session": session_name,
        "resolution": result["resolution"],
        "quality_analysis": quality_result
    })


In [101]:
with open("level3_quality_analysis.json", "w", encoding="utf-8") as f:
    json.dump(level3_results, f, ensure_ascii=False, indent=2)


## Level 4 - Action

In [102]:
REASON_MAP = {
    "قطع_تماس": "تماس به صورت ناگهانی قطع شده است",
    "تماس_ناتمام": "تماس بدون نتیجه پایان یافته است",
    "درخواست_اپراتور_انسانی": "کاربر درخواست ارتباط با نیروی انسانی داشت",
    "توافق_مرحله_اول": "کاندید برای مرحله اول تأیید شد",
    "زمان_مصاحبه_تعیین_شد": "زمان مصاحبه مشخص گردید"
}


In [103]:
def decide_action_from_result(resolution, score):
    if not resolution["resolved"]:
        return {
            "tool": "escalate_to_human_agent",
            "arguments": {
                "priority": "بالا",
                "reason": REASON_MAP.get(
                    resolution["resolution_signal"],
                    "نیاز به بررسی انسانی"
                )
            }
        }

    if score >= 7:
        return {
            "tool": "advance_candidate_stage",
            "arguments": {
                "next_stage": "مصاحبه_فنی"
            }
        }

    return {
        "tool": "follow_up_required",
        "arguments": {
            "reason": "کیفیت مصاحبه نیاز به پیگیری دارد"
        }
    }


In [ ]:
level4_results = []

for result in level3_results:
    quality = result.get("quality_analysis", {})

    score = quality.get("score")
    if score is None:
        score = 0

    action = decide_action_from_result(
        resolution=result["resolution"],
        score=score
    )

    level4_results.append({
        "session": result["session"],
        "action": action
    })


In [105]:
with open("level4_action_based_on_analysis.json", "w", encoding="utf-8") as f:
    json.dump(level4_results, f, ensure_ascii=False, indent=2)
